In [5]:
from database.MongoDBConnector import MongoDBConnector
from utils.data import process_row

import os
import json
from tqdm.auto import tqdm

from concurrent.futures import ProcessPoolExecutor, as_completed

### Set target 'add' or 'onset'

In [6]:
target = 'onset'

### Fetch data from MongoDB `model_data_$target`

In [7]:
mongo = MongoDBConnector(mode='remote')

add_data = await mongo.get_all_documents(
    coll_name=f"model_data_{target}",
    sort=[
        ('mobile', 1),
        ('measurement_date', 1)
    ]
)

print(len(add_data), "Measurements Fetched")

4537 Measurements Fetched


### Feature Extraction

In [8]:
dataset = []
with ProcessPoolExecutor(max_workers=max(1, os.cpu_count()-1)) as executor:

    futures = [executor.submit(process_row, row, target) for row in add_data]

    for fut in tqdm(as_completed(futures), total=len(futures)):

        rec = fut.result()
        if rec is not None:
            dataset.append(rec)

  0%|          | 0/4537 [00:00<?, ?it/s]

/Users/zhichao/Documents/E3A/ML/data-processing/data-processing/utils/helpers.py:15: RuntimeWarning: All-NaN slice encountered
  out[i] = np.nanmedian(w)
/Users/zhichao/Documents/E3A/ML/data-processing/data-processing/utils/helpers.py:15: RuntimeWarning: All-NaN slice encountered
  out[i] = np.nanmedian(w)
/Users/zhichao/Documents/E3A/ML/data-processing/data-processing/utils/helpers.py:15: RuntimeWarning: All-NaN slice encountered
  out[i] = np.nanmedian(w)
/Users/zhichao/Documents/E3A/ML/data-processing/data-processing/utils/helpers.py:15: RuntimeWarning: All-NaN slice encountered
  out[i] = np.nanmedian(w)
/Users/zhichao/Documents/E3A/ML/data-processing/data-processing/utils/helpers.py:15: RuntimeWarning: All-NaN slice encountered
  out[i] = np.nanmedian(w)
/Users/zhichao/Documents/E3A/ML/data-processing/data-processing/utils/helpers.py:15: RuntimeWarning: All-NaN slice encountered
  out[i] = np.nanmedian(w)
/Users/zhichao/Documents/E3A/ML/data-processing/data-processing/utils/helper

In [9]:
print(len(dataset), "Measurements")

with open(f'../datasets/dataset_{target}.json', 'w') as outfile:
    json.dump(dataset, outfile)

4006 Measurements
